In [69]:
import pandas as pd, numpy as np
from ortools.sat.python import cp_model

In [70]:
# entrada
nos = pd.read_excel('rotas_inputs.xlsx', sheet_name='nos')
caminhos = pd.read_excel('rotas_inputs.xlsx', sheet_name='caminhos')
n_nos = len(nos)
n_caminhos = len(caminhos)

In [71]:
# modelo
model = cp_model.CpModel()
x = np.zeros(n_caminhos).tolist()
for c in caminhos.index:
    # Caso duas dimensões f'x[{[x, y]}]
    x[c] = model.NewIntVar(0,1, f'x[{[c]}]')

In [72]:
# Função objetivo
model.Minimize(sum([x[c] * caminhos.distancia[c] for c in caminhos.index]))

In [73]:
# Restrições
# Define os nos de origem e destino
no_origem = int(nos.no[nos.desc=='origem'])
no_destino = int(nos.no[nos.desc=='destino'])

# Define os outros nos
model.Add(sum([x[c] for c in caminhos.index[caminhos.no_de==no_origem]])==1)
model.Add(sum([x[c] for c in caminhos.index[caminhos.no_para==no_destino]])==1)

# Somatório
for no in nos.no[nos.desc=='meio']:
    sum_entra = sum([x[c] for c in caminhos.index[caminhos.no_para==no]])
    sum_sai = sum([x[c] for c in caminhos.index[caminhos.no_de==no]])
    model.Add(sum_sai <= 1)
    model.Add(sum_entra <= 1)
    model.Add(sum_entra == sum_sai)

In [74]:
# Solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [79]:
# Impressão
print(f'Status = {solver.StatusName(status)}')
print(f'FO = {solver.ObjectiveValue()}')
caminhos['ativado'] = 0
for c in caminhos.index:
    caminhos.ativado[c] = solver.Value(x[c])
print(caminhos)

print('Rota escolhida')
for c in caminhos.index[caminhos.ativado==1]:
    print(f'X{caminhos.no_de[c]}{caminhos.no_para[c]} - {caminhos.distancia[c]}')

Status = OPTIMAL
FO = 1370.0
   no_de  no_para  distancia  ativado
0      1        2        220        1
1      1        3       1500        0
2      2        4        650        1
3      2        5        900        0
4      4        7        500        1
5      5        7        400        0
6      3        6        500        0
7      6        7        400        0
Rota escolhida
X12 - 220
X24 - 650
X47 - 500
